# Preparing Data

## Part 1: Get Data 

National Speech Corpus
- Part 3: 1000 hours of conversational speech data (Used by Home team)
- Part 2: 1000 hours of prompted recordings of random sentences containing local words and entities (Used by some developer)
- Part 4: Conversational code-switched data (from Singaporean English to various native languages)

<br/>
<br/>

- https://medium.com/htx-dsai/finetuning-whisper-for-the-singaporean-home-team-context-a3ae1a6ae809
- https://www.jensenlwt.com/blog/singlish-whisper-finetuning-asr-for-singapore-unique-english


<br/>
<br/>
<br/>
<br/>
<br/>

## Part 2: Prepare Data

- Match each transcript sentence to its corresponding audio file
- Check on the environment where the audio is recorded (decide the environment)
    - Hometeam
        - The NSC Part 3 recordings are split into two environments, each with two different microphones used for recording. In the first environment, where speakers were in the same room, we selected the recordings using the close-talk mic as this isolated the main speaker’s voice (without picking up background noise or the secondary speaker). For the second environment with speakers in different rooms, we chose to use the standing microphone recordings, as opposed to recordings via telephone.
    - Same room environment: Close-talk mic that isolates main speaker's voice 
    - Different room environment: Standing microphone as opposed to telephone
- Clean the transcripts by removing annotations
- Normalise the transcript text
    - Remove punctuations
    - Lowercase text
- Create 30s audio segments with corresponding transcripts
    - Using time segments from ```TextGrid files```, splice out corresponding segments from WAV files
    - Combine shorter consecutive segments (?)
    - 30s: Whisper's feature extractor ensures all audio is 30s (intrinsic design)

<br/>
<br/>

- https://medium.com/htx-dsai/finetuning-whisper-for-the-singaporean-home-team-context-a3ae1a6ae809
- https://www.jensenlwt.com/blog/singlish-whisper-finetuning-asr-for-singapore-unique-english

<br/>
<br/>

More on dataset part 3 (see ```ABOUT.txt```):

Part 3 consists of about 1000 hours of conversational data recorded from about 1000 local English speakers, split into pairs. The data includes conversations covering daily life and of speakers playing games provided. 

Part 3's recordings were split into 2 environments. In the Same Room environment where speakers were in same room, the recordings were done using 2 microphones: a close-talk mic and a boundary mic. In the Separate Room environment, speakers were separated into individual rooms. The recordings were done using 2 microphones in each room: a standing mic and a telephone. 

Part 3 is further organised into a six subdirectories, 3 for each recording environment (Same Room or Separate Room). Among each group of 3 subdirectories, 1 contains transcriptions, while the remaining 2 contain audio data from each of the two microphones used for the environment. There is also a manifest document at the root of the Part 3 folder that lists the files released.


Summary of Part 3 data organization:
- Same Room environment, files organized by speaker number:
    - /Scripts Same: Orthographic transcripts saved in TextGrid format
    - /Audio Same BoundaryMic: Audio files in WAV format recorded using the boundary mic, sampled at 16kHz
    - /Audio Same CloseMic: Audio files in WAV format recorded using the close-talk mic, sampled at 16kHz


- Separate Room environment, files organized by speaker number and session number:
    - /Scripts Separate: Orthographic transcripts saved in TextGrid format 
    - /Audio Separate IVR: Audio files in WAV format recorded using the telephone, sampled at 16kHz
    - /Audio Separate StandingMic: Audio files in WAV format recorded using the standing mic, sampled at 16kHz


<br/>
<br/>
<br/>
<br/>
<br/>

### **Iteration 1: Simple Example/Debugging**

**1. Match 3000-1.wav and 3000-1.TEXTGRID**

- Use Dataset Part 3 (used by Home Team)
- Specific datasets (used by Home Team)
    - Audio Same CloseMic
    - Audio Separate StandingMic 
- In this simple example, first settle the Audio Same CloseMic dataset


**2. Create 30s segments from 3000-1.wav and 3000-1.TEXTGRID**

In [ ]:
# https://github.com/jiaaro/pydub#installation
# https://github.com/timmahrt/praatIO/tree/main

import os
from praatio import textgrid 
from pydub import AudioSegment

# Initialise input and output paths
audio_path = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1.wav')
textgrid_path = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1.TextGrid')
output_dir = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1-splits')

# https://github.com/jiaaro/pydub
# https://github.com/timmahrt/praatIO
# https://timmahrt.github.io/praatIO/praatio.html
audio = AudioSegment.from_wav(audio_path)
tg = textgrid.openTextgrid(textgrid_path, False) # do not include intervals and points with empty labels

# pydub does things in milliseconds
segment_duration_ms = 30 * 1000  

# Get total duration of the audio in milliseconds
audio_duration = len(audio)

# Initialize start time and segment index
start_time = 0
segment_index = 1

#while start_time < audio_duration:
    # Initialise end time of the segment
end_time = min(start_time + segment_duration_ms, audio_duration)

# Extract audio segment given the current start and end timing
audio_segment = audio[start_time:end_time]

# Save the audio segment
audio_segment_path = os.path.join(output_dir, f'segment_{segment_index}.wav')
audio_segment.export(audio_segment_path, format="wav")

# Extract the corresponding TextGrid segment
# https://timmahrt.github.io/praatIO/praatio/data_classes/textgrid.html
tg_segment = tg.crop(start_time / 1000, end_time / 1000, mode="truncated", rebaseToZero=False)

# Check tg_segment 
# https://timmahrt.github.io/praatIO/praatio/data_classes/textgrid.html
tg_segment_path = os.path.join(output_dir, 'tg_segment.TextGrid')
tg_segment.save(tg_segment_path, "long_textgrid", True)

# Collect transcriptions from the TextGrid segment
transcriptions = []
for tier_name in tg_segment.tierNames: # For each tier (in order) in the TextGrid segment
    tier = tg_segment.getTier(tier_name) # Get the tier
    for entry in tier.entries: # For each of its entries, extract the labels 
        if entry.label.strip():  # Only include non-empty transcriptions -> but should be handled above already
            transcriptions.append(entry.label)

# Save the transcriptions to a text file
transcription_path = os.path.join(output_dir, f'segment_{segment_index}_transcription.txt')
with open(transcription_path, 'w') as f:
    f.write("\n".join(transcriptions))

In [ ]:
output_dir_audio = os.path.join(output_dir, 'segment_1.wav')

from IPython.display import Audio
display(Audio(output_dir_audio))

**Transcription**
```
<S>
(um) you can go first
<S>
you guys are going to stand here [ah]
<S>
they are like !wow! this is a weird topic (um)
<S>
Singapore and Malaysia are like
<S>
you know brothers but not really brothers brothers on a on a tricky relationship
<S>
you know what let's skip this topic
<S>
next do I go do I go next
```

**TextGrid**
```
File type = "ooTextFile"
Object class = "TextGrid"

xmin = 0 
xmax = 30 
tiers? <exists> 
size = 1 
item []: 
    item [1]:
        class = "IntervalTier" 
        name = "3000-1" 
        xmin = 0 
        xmax = 30 
        intervals: size = 14 
        intervals [1]:
            xmin = 0 
            xmax = 1.556 
            text = "<S>" 
        intervals [2]:
            xmin = 1.556 
            xmax = 2.661 
            text = "(um) you can go first" 
        intervals [3]:
            xmin = 2.661 
            xmax = 3.848 
            text = "<S>" 
        intervals [4]:
            xmin = 3.848 
            xmax = 4.998 
            text = "you guys are going to stand here [ah]" 
        intervals [5]:
            xmin = 4.998 
            xmax = 10.473 
            text = "<S>" 
        intervals [6]:
            xmin = 10.473 
            xmax = 13.531 
            text = "they are like !wow! this is a weird topic (um)" 
        intervals [7]:
            xmin = 13.531 
            xmax = 16.156 
            text = "<S>" 
        intervals [8]:
            xmin = 16.156 
            xmax = 17.868 
            text = "Singapore and Malaysia are like" 
        intervals [9]:
            xmin = 17.868 
            xmax = 19.781 
            text = "<S>" 
        intervals [10]:
            xmin = 19.781 
            xmax = 24.718 
            text = "you know brothers but not really brothers brothers on a on a tricky relationship" 
        intervals [11]:
            xmin = 24.718 
            xmax = 26.281 
            text = "<S>" 
        intervals [12]:
            xmin = 26.281 
            xmax = 27.318 
            text = "you know what let's skip this topic" 
        intervals [13]:
            xmin = 27.318 
            xmax = 28.156 
            text = "<S>" 
        intervals [14]:
            xmin = 28.156 
            xmax = 30 
            text = "next do I go do I go next" 

```

<br/>
<br/>
<br/>

**3. Clean and format the transcripts**

In [ ]:
output_dir_transcript = os.path.join(output_dir, 'segment_1_transcription.txt')

with open(output_dir_transcript, 'r') as f:
    transcript = ' '.join(line.strip() for line in f)

In [ ]:
transcript

<u>Cleaning</u>

1. Lower-case the text

2. Remove and replace annotations

- Acronyms: Remove '_'
- Multi-word nouns: Replace '-' with ' '
- Discourse particles: Remove '[' and ']'
- Fillers: Remove '(' and ')'
- Interjections: Remove '!'
- Paralinguistic Phenomena: Remove '(ppb)', '(ppc)', '(ppl)', '(ppo)'
- Other languages: Remove '#'
- Unclear words: Remove ```'<unk>'```
- Incomplete words: Remove '~'
- Short pauses: Remove ```'<s>'```
- Invalid: Remove ```'<z>'```
- Long-running non-english utterances: Remove ```'<nen>'```
- Fillers: Remove ```'<fil/>'```
- Speaker Noise: Remove ```'<spk/>'```
- Unknown: Remove '**'
- Non-primary speaker sound: Remove ```'<non/>'```
- End of sentence: Remove ```'<s/>'```
- Comma: Remove ```'<c/>'```

In [ ]:
import re

transcript = transcript.lower()

remove = [r'_', r'\[|\]', r'\(|\)', r'!', r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', 
          r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
          r'\*', r'<non/>', r'<s/>', r'<c/>']

replace = ['-']


for e in remove:
    transcript = re.sub(e, '', transcript)

In [ ]:
for e in replace:
    transcript = re.sub(e, ' ', transcript)

In [ ]:
transcript

In [ ]:
# Remove extra spaces created by <s> and stuff
transcript = re.sub(r'\s+', ' ', transcript).strip()

In [ ]:
transcript

**Need to change the order** 

(ppl) (ppb) etc. should be put infront because if the parantheses are removed, they won't be matched later

Also need to remove all ```<example_word>```, example: ```<malay>malay word</malay>```

In [ ]:
testing = ['(ppl)','(test)','sfs','(rdg)', 'tg_s']
testing_2 = ' '.join(test.strip() for test in testing)
remove = [r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', r'_', r'\[|\]', r'\(|\)', r'!', 
            r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
            r'\*', r'<non/>', r'<s/>', r'<c/>']
for e in remove:
    testing_2 = re.sub(e, '', testing_2)
testing_2 = re.sub(r'\s+', ' ', testing_2).strip()

In [ ]:
# https://github.com/jiaaro/pydub#installation
# https://github.com/timmahrt/praatIO/tree/main

import os
from praatio import textgrid 
from pydub import AudioSegment

# Initialise input and output paths
audio_path = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1.wav')
textgrid_path = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1.TextGrid')
output_dir = os.path.join(os.getcwd(), 'dataset', 'part3', 'simple_example', '3000-1-splits')

# https://github.com/jiaaro/pydub
# https://github.com/timmahrt/praatIO
# https://timmahrt.github.io/praatIO/praatio.html
audio = AudioSegment.from_wav(audio_path)
tg = textgrid.openTextgrid(textgrid_path, False) # do not include intervals and points with empty labels

# pydub does things in milliseconds
segment_duration_ms = 30 * 1000  

# Get total duration of the audio in milliseconds
audio_duration = len(audio)

# Initialize start time and segment index
start_time = 0
segment_index = 1

#while start_time < audio_duration:
    # Initialise end time of the segment
end_time = min(start_time + segment_duration_ms, audio_duration)

# Extract audio segment given the current start and end timing
audio_segment = audio[start_time:end_time]

# Save the audio segment
audio_segment_path = os.path.join(output_dir, f'segment_{segment_index}.wav')
audio_segment.export(audio_segment_path, format="wav")

# Extract the corresponding TextGrid segment
# https://timmahrt.github.io/praatIO/praatio/data_classes/textgrid.html
tg_segment = tg.crop(start_time / 1000, end_time / 1000, mode="truncated", rebaseToZero=False)

# Check tg_segment 
# https://timmahrt.github.io/praatIO/praatio/data_classes/textgrid.html
tg_segment_path = os.path.join(output_dir, 'tg_segment.TextGrid')
tg_segment.save(tg_segment_path, "long_textgrid", True)

# Collect transcriptions from the TextGrid segment
transcriptions = []
for tier_name in tg_segment.tierNames: # For each tier (in order) in the TextGrid segment
    tier = tg_segment.getTier(tier_name) # Get the tier
    for entry in tier.entries: # For each of its entries, extract the labels 
        if entry.label.strip():  # Only include non-empty transcriptions -> but should be handled above already
            transcriptions.append(entry.label)

print(transcriptions)

In [ ]:
' '.join(line.strip() for line in transcriptions)

In [ ]:
def clean_transcription(transcript):
    transcript = ' '.join(line.strip() for line in transcript)

    transcript = transcript.lower()

    remove = [r'_', r'\[|\]', r'\(|\)', r'!', r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', 
            r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
            r'\*', r'<non/>', r'<s/>', r'<c/>']

    replace = ['-']


    for e in remove:
        transcript = re.sub(e, '', transcript)

    for e in replace:
        transcript = re.sub(e, ' ', transcript)

    transcript = re.sub(r'\s+', ' ', transcript).strip()

    return transcript

In [ ]:
clean_transcription(transcriptions)

**Things to check**
- check out 3000-1_33: <malay>malay word</malay>
- check out 3000-1_36: no transcription

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

### **Iteration 2: Check if the transcriptions are still ok**

```
- imda_nsc_p3.tar.gz
    - imda_nsc_p3.tar
        - train
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-1_1.wav
                - 3000-1_2.wav
                - 3000-1_3.wav
- prompts-train.txt.gz
    - prompts-train.txt: Contains transcriptions for all the train .wav files
```

In [ ]:
import re 
import os
from praatio import textgrid 
from pydub import AudioSegment

In [ ]:
def clean_transcription(transcript):
    transcript = ' '.join(line.strip() for line in transcript)
    transcript = transcript.lower()
    remove = [r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', r'_', r'\[|\]', r'\(|\)', r'!', 
            r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
            r'\*', r'<non/>', r'<s/>', r'<c/>', r'<[^>]+>'] # Addition: remove all instances of <whatever's inside>
    replace = ['-']
    for e in remove:
        transcript = re.sub(e, '', transcript)
    for e in replace:
        transcript = re.sub(e, ' ', transcript)
    transcript = re.sub(r'\s+', ' ', transcript).strip()
    return transcript

In [ ]:
# Input paths
audio_filename = '3000-2'

audio_path = os.path.join(os.getcwd(), 'dataset', 'dev', 'org_wavs', f'{audio_filename}.wav')
textgrid_path = os.path.join(os.getcwd(), 'dataset', 'dev', 'org_transcripts', f'{audio_filename}.TextGrid')

# Output paths
# output_dir_train_wav = os.path.join(os.getcwd(), 'dataset', 'imda_nsc_prototype', 'train', 'waves', f'{audio_filename}')
output_dir_train_wav = os.path.join(os.getcwd(), 'dataset', 'dev', 'train', 'waves')
os.makedirs(output_dir_train_wav, exist_ok=True)
output_dir_train_text = os.path.join(os.getcwd(), 'dataset', 'dev', 'train', 'prompts.txt')
output_dir_train_tg = os.path.join(os.getcwd(), 'dataset', 'dev', 'train', 'textgrids')

# https://github.com/jiaaro/pydub
# https://github.com/timmahrt/praatIO
# https://timmahrt.github.io/praatIO/praatio.html
# Extract the audio and text grid
audio = AudioSegment.from_wav(audio_path)
tg = textgrid.openTextgrid(textgrid_path, False) # do not include intervals and points with empty labels

# pydub does things in milliseconds
segment_duration_ms = 30 * 1000  

# Get total duration of the audio in milliseconds
audio_duration = len(audio)

# Initialize start time and segment index
start_time = 0
segment_index = 1

while start_time < audio_duration:
    # Initialise end time of the segment
    end_time = min(start_time + segment_duration_ms, audio_duration)

    # Extract audio segment given the current start and end timing
    audio_segment = audio[start_time:end_time]

    # Extract the corresponding TextGrid segment
    # https://timmahrt.github.io/praatIO/praatio/data_classes/textgrid.html
    tg_segment = tg.crop(start_time / 1000, end_time / 1000, mode="truncated", rebaseToZero=False)

    tg_segment_path = os.path.join(output_dir_train_tg, f'{audio_filename}_{segment_index}.TextGrid')
    tg_segment.save(tg_segment_path, "long_textgrid", True)

    # Collect transcriptions from the TextGrid segment
    transcriptions = []
    for tier_name in tg_segment.tierNames: # For each tier (in order) in the TextGrid segment
        tier = tg_segment.getTier(tier_name) # Get the tier
        for entry in tier.entries: # For each of its entries, extract the labels 
            if entry.label.strip():  # Only include non-empty transcriptions -> but should be handled above already
                transcriptions.append(entry.label)

    print(f"Dirty transcription: {transcriptions}")
    # Clean the transcriptions
    transcriptions_clean = clean_transcription(transcriptions)
    print(f"Clean transcription: {transcriptions_clean}")
    #print("")

    if len(transcriptions_clean) > 0:
        # Save the transcriptions to a text file, append mode
        with open(output_dir_train_text, 'a') as f:
            f.write(f'{audio_filename}_{segment_index} {transcriptions_clean}\n')

        # Save the audio segment
        audio_segment_path = os.path.join(output_dir_train_wav, f'{audio_filename}_{segment_index}.wav')
        audio_segment.export(audio_segment_path, format="wav")

        start_time+=segment_duration_ms
        segment_index+=1
    else:
        start_time+=segment_duration_ms

**```tar.gz``` file resources**

- https://stackoverflow.com/questions/2032403/how-to-create-full-compressed-tar-file-using-python
- https://www.tutorialspoint.com/how-to-create-a-tar-file-using-python
- https://www.geeksforgeeks.org/python-os-path-relpath-method/

**```txt.gz file resources```**
- https://stackoverflow.com/questions/8156707/gzip-a-file-in-python

**```folder structure resources```**
- https://huggingface.co/docs/datasets/en/audio_dataset#loading-script
- https://huggingface.co/datasets/AILAB-VNUHCM/vivos/tree/main/data

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

### **Iteration 3: Testing**

<u>Before running processing code</u>
```
dataset
- testing
    - data: Empty
    - org_waves: Manually add in .wav files
        - 3000-1.wav
        - 3000-2.wav
        - ...
    - org_transcripts: Manually add in .TextGrid files
        - 3000-1.TextGrid
        - 3000-2.TextGrid
        - ...
    - train
        - waves: Empty
        - transcripts: Empty
    - test
        - waves: Empty
        - transcripts: Empty
```

<br/>

<u>After running processing code</u>
```
dataset
- testing
    - data: Empty
    - org_waves: Manually add in .wav files
        - 3000-1.wav
        - 3000-2.wav
        - ...
    - org_transcripts: Manually add in .TextGrid files
        - 3000-1.TextGrid
        - 3000-2.TextGrid
        - ...
    - train
        - prompts.txt: Contains transcriptions for all the .wav files in train
        - waves
            - 3000-1_1.wav
            - 3000-1_2.wav
            - 3000-1_3.wav
            - ...
            - 3000-2_1.wav
            - 3000-2_2.wav
            - 3000-2_3.wav
        - transcripts
            - 3000-1_1.txt
            - 3000-1_2.txt
            - 3000-1_3.txt
            - ...
            - 3000-2_1.txt
            - 3000-2_2.txt
            - 3000-2_3.txt
    - test
        - prompts.txt: Contains transcriptions for all the .wav files in test
        - waves
            - 3000-3_1.wav
            - 3000-3_2.wav
            - 3000-3_3.wav
            - ...
            - 3000-4_1.wav
            - 3000-4_2.wav
            - 3000-4_3.wav
        - transcripts
            - 3000-3_1.txt
            - 3000-3_2.txt
            - 3000-3_3.txt
            - ...
            - 3000-4_1.txt
            - 3000-4_2.txt
            - 3000-4_3.txt
```

<br/>

<u>After running compression code</u>

```
data
    - input_name.tar.gz
        - train
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-1_1.wav
                - 3000-1_2.wav
                - 3000-1_3.wav
                - ...
                - 3000-2_1.wav
                - 3000-2_2.wav
                - 3000-2_3.wav
        - test
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-3_1.wav
                - 3000-3_2.wav
                - 3000-3_3.wav
                - ...
                - 3000-4_1.wav
                - 3000-4_2.wav
                - 3000-4_3.wav
    - prompts-train.txt.gz
        - prompts-train.txt: Contains transcriptions for all the train .wav files -> take this from train/prompts.txt
    - prompts-test.txt.gz
        - prompts-test.txt: Contains transcriptions for all the test .wav files -> take this from test/prompts.txt
```

**Imports**

In [ ]:
import re 
import os
import shutil
import tarfile
import gzip
import math
import random
from praatio import textgrid 
from pydub import AudioSegment

c:\Users\johnl\miniconda3\envs\myenv2\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


**Input Relative Paths**

In [ ]:
input_audio_path = ['dataset', 'testing', 'org_wavs']
input_textgrid_path = ['dataset', 'testing', 'org_transcripts']
output_train_path = ['dataset', 'testing', 'train']
output_test_path = ['dataset', 'testing', 'test']
output_compressed_path = ['dataset', 'testing']
compressed_filename = 'imda_nsc_p3_testing.tar.gz'
compressed_train_prompt_filename = 'prompts-train.txt.gz'
compressed_test_prompt_filename = 'prompts-test.txt.gz'

**Initialise Paths and Create the directories**

**IMPT**: Remember to add in the ```.wav``` and ```.TextGrid``` files to org_waves and org_transcripts

In [ ]:
input_wav_folder = os.path.join(os.getcwd(), *input_audio_path)
input_textgrid_folder = os.path.join(os.getcwd(), *input_textgrid_path)
output_train_folder_waves = os.path.join(os.getcwd(), *output_train_path, 'waves')
output_train_folder_transcripts = os.path.join(os.getcwd(), *output_train_path, 'transcripts')
output_test_folder_waves  = os.path.join(os.getcwd(), *output_test_path, 'waves')
output_test_folder_transcripts = os.path.join(os.getcwd(), *output_test_path, 'transcripts')
output_textgrids_folder = os.path.join(os.getcwd(), *output_train_path, 'textgrids')
output_compressed_folder = os.path.join(os.getcwd(), *output_compressed_path, 'data')
output_compressed_file = os.path.join(output_compressed_folder, compressed_filename)
output_compressed_train_prompt_file = os.path.join(output_compressed_folder, compressed_train_prompt_filename)
output_compressed_test_prompt_file = os.path.join(output_compressed_folder, compressed_test_prompt_filename)

create_dir = [input_wav_folder, input_textgrid_folder, output_train_folder_waves, output_train_folder_transcripts,
              output_test_folder_waves, output_test_folder_transcripts, output_textgrids_folder, output_compressed_folder]

for dir in create_dir:
    os.makedirs(dir, exist_ok=True)

**Helper function to clean the transcription**

1. Lower-case the text

2. Remove and replace annotations

- Paralinguistic Phenomena: Remove '(ppb)', '(ppc)', '(ppl)', '(ppo)'
- Acronyms: Remove '_'
- Multi-word nouns: Replace '-' with ' '
- Discourse particles: Remove '[' and ']'
- Fillers: Remove '(' and ')'
- Interjections: Remove '!'
- Other languages: Remove '#'
- Unclear words: Remove ```'<unk>'```
- Incomplete words: Remove '~'
- Short pauses: Remove ```'<s>'```
- Invalid: Remove ```'<z>'```
- Long-running non-english utterances: Remove ```'<nen>'```
- Fillers: Remove ```'<fil/>'```
- Speaker Noise: Remove ```'<spk/>'```
- Unknown: Remove '**'
- Non-primary speaker sound: Remove ```'<non/>'```
- End of sentence: Remove ```'<s/>'```
- Comma: Remove ```'<c/>'```
- Remove all instances of ```<whatever is inside>```

3. Remove extra spaces created by ```<s>``` and stuff

Refer to the Transcription Guidelines by IMDA

In [ ]:
def clean_transcription(transcript):
    transcript = ' '.join(line.strip() for line in transcript)
    transcript = transcript.lower()
    remove = [r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', r'_', r'\[|\]', r'\(|\)', r'!', 
            r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
            r'\*', r'<non/>', r'<s/>', r'<c/>', r'<[^>]+>'] 
    replace = ['-']
    for e in remove:
        transcript = re.sub(e, '', transcript)
    for e in replace:
        transcript = re.sub(e, ' ', transcript)
    transcript = re.sub(r'\s+', ' ', transcript).strip()
    return transcript

**Main function**

Matches a single ```.wav``` file to its respective ```.TextGrid``` file

- Break the ```.wav``` file and ```.TextGrid``` file into 30s segments
- Clean the ```.TextGrid``` file
- Only keep segments that have audio

In [ ]:
def process_audio_transcript(audio_filename, input_audio_path, input_textgrid_path, output_path, sanity_check=False):
    audio_path = os.path.join(os.getcwd(), *input_audio_path, f'{audio_filename}.wav')
    textgrid_path = os.path.join(os.getcwd(), *input_textgrid_path, f'{audio_filename}.TextGrid')

    output_dir_wav = os.path.join(os.getcwd(), *output_path, 'waves')
    output_dir_transcript = os.path.join(os.getcwd(), *output_path, 'transcripts')

    output_dir_textgrid = os.path.join(os.getcwd(), *output_path, 'textgrids')

    audio = AudioSegment.from_wav(audio_path)
    tg = textgrid.openTextgrid(textgrid_path, False) 

    segment_duration_ms = 30 * 1000  

    audio_duration = len(audio)

    start_time = 0
    segment_index = 1

    while start_time < audio_duration:
        end_time = min(start_time + segment_duration_ms, audio_duration)

        audio_segment = audio[start_time:end_time]
        tg_segment = tg.crop(start_time / 1000, end_time / 1000, mode="truncated", rebaseToZero=False)

        transcriptions = []
        for tier_name in tg_segment.tierNames: 
            tier = tg_segment.getTier(tier_name) 
            for entry in tier.entries:  
                if entry.label.strip():  
                    transcriptions.append(entry.label)

        transcriptions_clean = clean_transcription(transcriptions)

        if len(transcriptions_clean) > 0:
            transcript_segment_path = os.path.join(output_dir_transcript, f'{audio_filename}_{segment_index}.txt')
            with open(transcript_segment_path, 'w') as f:
                f.write(f'{audio_filename}_{segment_index} {transcriptions_clean}')

            if sanity_check:
                tg_segment_path = os.path.join(output_dir_textgrid, f'{audio_filename}_{segment_index}.TextGrid')
                tg_segment.save(tg_segment_path, "long_textgrid", True)
            
            audio_segment_path = os.path.join(output_dir_wav, f'{audio_filename}_{segment_index}.wav')
            audio_segment.export(audio_segment_path, format="wav")

            start_time+=segment_duration_ms
            segment_index+=1
        else:
            start_time+=segment_duration_ms

**Run the main function to segment 30s chunks for each ```.wav``` and ```.TextGrid``` file**

Output is the segmented ```.wav``` files and transcriptions for each ```.wav``` file stored in ```train/waves``` and ```train/transcripts``` respectively

Note: We first put the files into the train folder

A sanity check can be set to True to view the segmented ```.TextGrid``` files in ```./train/textgrids/```

In [ ]:
audio_path = os.path.join(os.getcwd(), *input_audio_path)
for filename in os.listdir(audio_path):
    filename = filename.split('.')[0]
    process_audio_transcript(filename, input_audio_path, input_textgrid_path, output_train_path, True)

**Move a split of the ```.wav``` files and ```.txt``` file to test**

In [ ]:
test_split = 0.2

sample_filenames = []
for filename in os.listdir(output_train_folder_waves):
    sample_filenames.append(filename.split('.')[0])

samples = len(sample_filenames)

num_train_samples = math.floor((1-test_split)*samples)
num_test_samples = samples-num_train_samples

print(f"The total number of samples is {samples}")
print(f"The total number of training samples will be {num_train_samples}")
print(f"The total number of test samples will be {num_test_samples}")

The total number of samples is 238
The total number of training samples will be 190
The total number of test samples will be 48


In [ ]:
random.shuffle(sample_filenames)

In [ ]:
for i in range(num_test_samples):
    filename = sample_filenames[i]

    source_wav = os.path.join(output_train_folder_waves, filename + '.wav')
    destination_wav = os.path.join(output_test_folder_waves)
    shutil.move(source_wav, destination_wav)

    source_transcript = os.path.join(output_train_folder_transcripts, filename + '.txt')
    destination_transcript = os.path.join(output_test_folder_transcripts)
    shutil.move(source_transcript, destination_transcript)

**Write the ```/train/prompts.txt``` and ```/test/prompts.txt``` files**

In [ ]:
train_prompts_path = os.path.join(os.getcwd(), *output_train_path, 'prompts.txt')
with open(train_prompts_path, 'a') as outfile:
    for filename in os.listdir(output_train_folder_transcripts):
        file_path = os.path.join(output_train_folder_transcripts, filename)
        with open(file_path, "r") as infile:
            outfile.write(infile.read() + '\n')

In [ ]:
test_prompts_path = os.path.join(os.getcwd(), *output_test_path, 'prompts.txt')
with open(test_prompts_path, 'a') as outfile:
    for filename in os.listdir(output_test_folder_transcripts):
        file_path = os.path.join(output_test_folder_transcripts, filename)
        with open(file_path, "r") as infile:
            outfile.write(infile.read() + '\n')

**Compress the folders into ```.tar.gzip```**

In [ ]:
paths_to_compress = [train_prompts_path, output_train_folder_waves, test_prompts_path, output_test_folder_waves]

with tarfile.open(output_compressed_file, "w:gz") as tar_gz:
    for path in paths_to_compress:
        rel_path = os.path.relpath(path, os.path.join(os.getcwd(), *output_compressed_path))
        tar_gz.add(path, arcname=rel_path) 

In [ ]:
with open(train_prompts_path, 'rb') as f_in, gzip.open(output_compressed_train_prompt_file, 'wb') as f_out:
    f_out.writelines(f_in)

In [ ]:
with open(test_prompts_path, 'rb') as f_in, gzip.open(output_compressed_test_prompt_file, 'wb') as f_out:
    f_out.writelines(f_in)

**Sanity Check**

In [ ]:
with open(train_prompts_path, "r") as f:
    lines = f.readlines()
    train_prompts_filenames = sorted([l.split(' ')[0] for l in lines])

In [ ]:
train_prompts_filenames[:10]

['3000-1_1',
 '3000-1_100',
 '3000-1_101',
 '3000-1_102',
 '3000-1_103',
 '3000-1_104',
 '3000-1_105',
 '3000-1_106',
 '3000-1_107',
 '3000-1_108']

In [ ]:
train_wavs_filenames = []
for filename in os.listdir(output_train_folder_waves):
    filename = filename.split('.')[0]
    train_wavs_filenames.append(filename)
train_waves_filename = sorted(train_wavs_filenames)

In [ ]:
train_waves_filename[:10]

['3000-1_1',
 '3000-1_100',
 '3000-1_101',
 '3000-1_102',
 '3000-1_103',
 '3000-1_104',
 '3000-1_105',
 '3000-1_106',
 '3000-1_107',
 '3000-1_108']

In [ ]:
train_prompts_filenames==train_waves_filename

True

In [ ]:
with open(test_prompts_path, "r") as f:
    lines = f.readlines()
    test_prompts_filenames = sorted([l.split(' ')[0] for l in lines])

In [ ]:
test_prompts_filenames[:10]

['3000-1_10',
 '3000-1_118',
 '3000-1_119',
 '3000-1_13',
 '3000-1_15',
 '3000-1_27',
 '3000-1_28',
 '3000-1_32',
 '3000-1_33',
 '3000-1_34']

In [ ]:
test_wavs_filenames = []
for filename in os.listdir(output_test_folder_waves):
    filename = filename.split('.')[0]
    test_wavs_filenames.append(filename)
test_waves_filename = sorted(test_wavs_filenames)

In [ ]:
test_wavs_filenames[:10]

['3000-1_10',
 '3000-1_118',
 '3000-1_119',
 '3000-1_13',
 '3000-1_15',
 '3000-1_27',
 '3000-1_28',
 '3000-1_32',
 '3000-1_33',
 '3000-1_34']

In [ ]:
test_prompts_filenames==test_wavs_filenames

True

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

### **Iteration 4: How to fix overlap between some audio files and transcriptions**

Example: 

3000-1_12 and 3000-1_13

Audio at the end of 3000-1_12 includes 3/4-ish of text in ```intervals[22]```

Audio at the start of 3000-1_13 includes 1/4-ish of text in ```intervals[1]```

Solution: Segment based on TextGrid files instead of Audio files?

### Step 1:
<u>Initialising the directory</u>
```
dataset
- testing
    - data: Used to store compression files
    - org_waves: Manually add in .wav files
        - 3000-1.wav
        - 3000-2.wav
        - ...
    - org_transcripts: Manually add in .TextGrid files
        - 3000-1.TextGrid
        - 3000-2.TextGrid
        - ...
    - train
        - waves: Empty
        - transcripts: Empty
    - test
        - waves: Empty
        - transcripts: Empty
```

<br/>

### Step 2:
<u>After running the processing code</u>
```
dataset
- testing
    - data: Used to store compression files
    - org_waves: Manually add in .wav files
        - 3000-1.wav
        - 3000-2.wav
        - ...
    - org_transcripts: Manually add in .TextGrid files
        - 3000-1.TextGrid
        - 3000-2.TextGrid
        - ...
    - train
        - prompts.txt: Contains transcriptions for all the .wav files in train
        - waves
            - 3000-1_1.wav
            - 3000-1_2.wav
            - 3000-1_3.wav
            - ...
            - 3000-2_1.wav
            - 3000-2_2.wav
            - 3000-2_3.wav
        - transcripts
            - 3000-1_1.txt
            - 3000-1_2.txt
            - 3000-1_3.txt
            - ...
            - 3000-2_1.txt
            - 3000-2_2.txt
            - 3000-2_3.txt
    - test
        - prompts.txt: Contains transcriptions for all the .wav files in test
        - waves
            - 3000-3_1.wav
            - 3000-3_2.wav
            - 3000-3_3.wav
            - ...
            - 3000-4_1.wav
            - 3000-4_2.wav
            - 3000-4_3.wav
        - transcripts
            - 3000-3_1.txt
            - 3000-3_2.txt
            - 3000-3_3.txt
            - ...
            - 3000-4_1.txt
            - 3000-4_2.txt
            - 3000-4_3.txt
```

<br/>

### Step 3:
<u>After running the compression code</u>

```
data
    - input_name.tar.gz
        - train
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-1_1.wav
                - 3000-1_2.wav
                - 3000-1_3.wav
                - ...
                - 3000-2_1.wav
                - 3000-2_2.wav
                - 3000-2_3.wav
        - test
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-3_1.wav
                - 3000-3_2.wav
                - 3000-3_3.wav
                - ...
                - 3000-4_1.wav
                - 3000-4_2.wav
                - 3000-4_3.wav
    - prompts-train.txt.gz
        - prompts-train.txt: Contains transcriptions for all the train .wav files -> take this from train/prompts.txt
    - prompts-test.txt.gz
        - prompts-test.txt: Contains transcriptions for all the test .wav files -> take this from test/prompts.txt
```

**Imports**

In [ ]:
import re 
import os
import shutil
import tarfile
import gzip
import math
import random
from praatio import textgrid 
from pydub import AudioSegment

c:\Users\johnl\miniconda3\envs\myenv2\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


**<u>USER INPUT REQUIRED</u> Input Relative Paths**

In [ ]:
input_audio_path = ['dataset', 'testing', 'org_wavs']
input_textgrid_path = ['dataset', 'testing', 'org_transcripts']
output_train_path = ['dataset', 'testing', 'train']
output_test_path = ['dataset', 'testing', 'test']
output_compressed_path = ['dataset', 'testing']
compressed_filename = 'imda_nsc_p3_testing.tar.gz'
compressed_train_prompt_filename = 'prompts-train.txt.gz'
compressed_test_prompt_filename = 'prompts-test.txt.gz'

**Initialise Paths and Create the directories**

**IMPT <u>USER INPUT REQUIRED</u>**: Remember to add in the ```.wav``` and ```.TextGrid``` files to org_waves and org_transcripts

In [ ]:
input_wav_folder = os.path.join(os.getcwd(), *input_audio_path)
input_textgrid_folder = os.path.join(os.getcwd(), *input_textgrid_path)
output_train_folder_waves = os.path.join(os.getcwd(), *output_train_path, 'waves')
output_train_folder_transcripts = os.path.join(os.getcwd(), *output_train_path, 'transcripts')
output_test_folder_waves  = os.path.join(os.getcwd(), *output_test_path, 'waves')
output_test_folder_transcripts = os.path.join(os.getcwd(), *output_test_path, 'transcripts')
output_textgrids_folder = os.path.join(os.getcwd(), *output_train_path, 'textgrids')
output_compressed_folder = os.path.join(os.getcwd(), *output_compressed_path, 'data')
output_compressed_file = os.path.join(output_compressed_folder, compressed_filename)
output_compressed_train_prompt_file = os.path.join(output_compressed_folder, compressed_train_prompt_filename)
output_compressed_test_prompt_file = os.path.join(output_compressed_folder, compressed_test_prompt_filename)

create_dir = [input_wav_folder, input_textgrid_folder, output_train_folder_waves, output_train_folder_transcripts,
              output_test_folder_waves, output_test_folder_transcripts, output_textgrids_folder, output_compressed_folder]

for dir in create_dir:
    os.makedirs(dir, exist_ok=True)

**Helper function to clean the transcription**

1. Lower-case the text

2. Remove and replace annotations

- Paralinguistic Phenomena: Remove '(ppb)', '(ppc)', '(ppl)', '(ppo)'
- Acronyms: Remove '_'
- Multi-word nouns: Replace '-' with ' '
- Discourse particles: Remove '[' and ']'
- Fillers: Remove '(' and ')'
- Interjections: Remove '!'
- Other languages: Remove '#'
- Unclear words: Remove ```'<unk>'```
- Incomplete words: Remove '~'
- Short pauses: Remove ```'<s>'```
- Invalid: Remove ```'<z>'```
- Long-running non-english utterances: Remove ```'<nen>'```
- Fillers: Remove ```'<fil/>'```
- Speaker Noise: Remove ```'<spk/>'```
- Unknown: Remove '**'
- Non-primary speaker sound: Remove ```'<non/>'```
- End of sentence: Remove ```'<s/>'```
- Comma: Remove ```'<c/>'```
- Remove all instances of ```<whatever is inside>```

3. Remove extra spaces created by ```<s>``` and stuff

Refer to the Transcription Guidelines by IMDA

In [ ]:
def clean_transcription(transcript):
    transcript = ' '.join(line.strip() for line in transcript)
    transcript = transcript.lower()
    remove = [r'\(ppb\)|\(ppc\)|\(ppl\)|\(ppo\)', r'_', r'\[|\]', r'\(|\)', r'!', 
            r'#', r'<unk>', r'~', r'<s>', r'<z>', r'<nen>', r'<fil/>', r'<spk/>',
            r'\*', r'<non/>', r'<s/>', r'<c/>', r'<[^>]+>'] 
    replace = ['-']
    for e in remove:
        transcript = re.sub(e, '', transcript)
    for e in replace:
        transcript = re.sub(e, ' ', transcript)
    transcript = re.sub(r'\s+', ' ', transcript).strip()
    return transcript

**Main function**

Matches a single ```.wav``` file to its respective ```.TextGrid``` file

- Break the ```.wav``` file and ```.TextGrid``` file into 30s segments
- Clean the ```.TextGrid``` file
- Only keep segments that have audio

In [ ]:
def process_audio_transcript(audio_filename, input_audio_path, input_textgrid_path, output_path, sanity_check=False):
    audio_path = os.path.join(os.getcwd(), *input_audio_path, f'{audio_filename}.wav')
    textgrid_path = os.path.join(os.getcwd(), *input_textgrid_path, f'{audio_filename}.TextGrid')

    output_dir_wav = os.path.join(os.getcwd(), *output_path, 'waves')
    output_dir_transcript = os.path.join(os.getcwd(), *output_path, 'transcripts')

    output_dir_textgrid = os.path.join(os.getcwd(), *output_path, 'textgrids')

    audio = AudioSegment.from_wav(audio_path)
    tg = textgrid.openTextgrid(textgrid_path, False) 

    segment_duration_ms = 30 * 1000  

    audio_duration = len(audio)

    start_time = 0
    segment_index = 1

    while start_time < audio_duration:
        end_time = min(start_time + segment_duration_ms, audio_duration)

        audio_segment = audio[start_time:end_time]
        tg_segment = tg.crop(start_time / 1000, end_time / 1000, mode="truncated", rebaseToZero=False)

        transcriptions = []
        for tier_name in tg_segment.tierNames: 
            tier = tg_segment.getTier(tier_name) 
            for entry in tier.entries:  
                if entry.label.strip():  
                    transcriptions.append(entry.label)

        transcriptions_clean = clean_transcription(transcriptions)

        if len(transcriptions_clean) > 0:
            transcript_segment_path = os.path.join(output_dir_transcript, f'{audio_filename}_{segment_index}.txt')
            with open(transcript_segment_path, 'w') as f:
                f.write(f'{audio_filename}_{segment_index} {transcriptions_clean}')

            if sanity_check:
                tg_segment_path = os.path.join(output_dir_textgrid, f'{audio_filename}_{segment_index}.TextGrid')
                tg_segment.save(tg_segment_path, "long_textgrid", True)
            
            audio_segment_path = os.path.join(output_dir_wav, f'{audio_filename}_{segment_index}.wav')
            audio_segment.export(audio_segment_path, format="wav")

            start_time+=segment_duration_ms
            segment_index+=1
        else:
            start_time+=segment_duration_ms

**Run the main function to segment 30s chunks for each ```.wav``` and ```.TextGrid``` file**

Output is the segmented ```.wav``` files and transcriptions for each ```.wav``` file stored in ```train/waves``` and ```train/transcripts``` respectively

Note: We first put the files into the train folder

A sanity check can be set to True to view the segmented ```.TextGrid``` files in ```./train/textgrids/```

In [ ]:
audio_path = os.path.join(os.getcwd(), *input_audio_path)
for filename in os.listdir(audio_path):
    filename = filename.split('.')[0]
    process_audio_transcript(filename, input_audio_path, input_textgrid_path, output_train_path, True)

**Move a split of the ```.wav``` files and ```.txt``` file to test**

In [ ]:
test_split = 0.2

sample_filenames = []
for filename in os.listdir(output_train_folder_waves):
    sample_filenames.append(filename.split('.')[0])

samples = len(sample_filenames)

num_train_samples = math.floor((1-test_split)*samples)
num_test_samples = samples-num_train_samples

print(f"The total number of samples is {samples}")
print(f"The total number of training samples will be {num_train_samples}")
print(f"The total number of test samples will be {num_test_samples}")

The total number of samples is 238
The total number of training samples will be 190
The total number of test samples will be 48


In [ ]:
random.shuffle(sample_filenames)

In [ ]:
for i in range(num_test_samples):
    filename = sample_filenames[i]

    source_wav = os.path.join(output_train_folder_waves, filename + '.wav')
    destination_wav = os.path.join(output_test_folder_waves)
    shutil.move(source_wav, destination_wav)

    source_transcript = os.path.join(output_train_folder_transcripts, filename + '.txt')
    destination_transcript = os.path.join(output_test_folder_transcripts)
    shutil.move(source_transcript, destination_transcript)

**Write the ```/train/prompts.txt``` and ```/test/prompts.txt``` files**

In [ ]:
train_prompts_path = os.path.join(os.getcwd(), *output_train_path, 'prompts.txt')
with open(train_prompts_path, 'a') as outfile:
    for filename in os.listdir(output_train_folder_transcripts):
        file_path = os.path.join(output_train_folder_transcripts, filename)
        with open(file_path, "r") as infile:
            outfile.write(infile.read() + '\n')

In [ ]:
test_prompts_path = os.path.join(os.getcwd(), *output_test_path, 'prompts.txt')
with open(test_prompts_path, 'a') as outfile:
    for filename in os.listdir(output_test_folder_transcripts):
        file_path = os.path.join(output_test_folder_transcripts, filename)
        with open(file_path, "r") as infile:
            outfile.write(infile.read() + '\n')

**Compress the folders into ```.tar.gzip```**

In [ ]:
paths_to_compress = [train_prompts_path, output_train_folder_waves, test_prompts_path, output_test_folder_waves]

with tarfile.open(output_compressed_file, "w:gz") as tar_gz:
    for path in paths_to_compress:
        rel_path = os.path.relpath(path, os.path.join(os.getcwd(), *output_compressed_path))
        tar_gz.add(path, arcname=rel_path) 

In [ ]:
with open(train_prompts_path, 'rb') as f_in, gzip.open(output_compressed_train_prompt_file, 'wb') as f_out:
    f_out.writelines(f_in)

In [ ]:
with open(test_prompts_path, 'rb') as f_in, gzip.open(output_compressed_test_prompt_file, 'wb') as f_out:
    f_out.writelines(f_in)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

**Sanity Check**

In [ ]:
with open(train_prompts_path, "r") as f:
    lines = f.readlines()
    train_prompts_filenames = sorted([l.split(' ')[0] for l in lines])

In [ ]:
train_prompts_filenames[:10]

['3000-1_1',
 '3000-1_100',
 '3000-1_101',
 '3000-1_102',
 '3000-1_103',
 '3000-1_104',
 '3000-1_105',
 '3000-1_106',
 '3000-1_107',
 '3000-1_108']

In [ ]:
train_wavs_filenames = []
for filename in os.listdir(output_train_folder_waves):
    filename = filename.split('.')[0]
    train_wavs_filenames.append(filename)
train_waves_filename = sorted(train_wavs_filenames)

In [ ]:
train_waves_filename[:10]

['3000-1_1',
 '3000-1_100',
 '3000-1_101',
 '3000-1_102',
 '3000-1_103',
 '3000-1_104',
 '3000-1_105',
 '3000-1_106',
 '3000-1_107',
 '3000-1_108']

In [ ]:
train_prompts_filenames==train_waves_filename

True

In [ ]:
with open(test_prompts_path, "r") as f:
    lines = f.readlines()
    test_prompts_filenames = sorted([l.split(' ')[0] for l in lines])

In [ ]:
test_prompts_filenames[:10]

['3000-1_10',
 '3000-1_118',
 '3000-1_119',
 '3000-1_13',
 '3000-1_15',
 '3000-1_27',
 '3000-1_28',
 '3000-1_32',
 '3000-1_33',
 '3000-1_34']

In [ ]:
test_wavs_filenames = []
for filename in os.listdir(output_test_folder_waves):
    filename = filename.split('.')[0]
    test_wavs_filenames.append(filename)
test_waves_filename = sorted(test_wavs_filenames)

In [ ]:
test_wavs_filenames[:10]

['3000-1_10',
 '3000-1_118',
 '3000-1_119',
 '3000-1_13',
 '3000-1_15',
 '3000-1_27',
 '3000-1_28',
 '3000-1_32',
 '3000-1_33',
 '3000-1_34']

In [ ]:
test_prompts_filenames==test_wavs_filenames

True

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## Part 3: Upload to HF

<u>Upload to HuggingFace</u>

Prepare our own audio dataset and upload it to HF

Stream data during the training process

Each file is around 112770 KB which is 0.11 GB

Part 3 consists of 1000 hours, which is maybe 110 GB ish

But maybe half of it is not the enviornment we want

<br/>
<br/>

Folder structure

Configure your dataset repository with audio files

- https://huggingface.co/docs/datasets/audio_dataset#audiofolder
- https://huggingface.co/docs/datasets/en/repository_structure#split-pattern-hierarchy
- https://huggingface.co/docs/hub/datasets-audio

```
test_dataset
    - metadata.csv: file_name (full relative path to audio file), transcription
    - data
        - train
            - first_train_audio_file.wav
            - second_train_audio_file.wav
            - ...
```

<br/>
<br/>
<br/>
<br/>
<br/>


### <u>Approach 1</u>

**<u>Part 1: Folder-based builders: Build dataset locally</u>**

https://huggingface.co/docs/datasets/create_dataset

https://huggingface.co/docs/datasets/audio_dataset#audiofolder

https://huggingface.co/docs/datasets/en/repository_structure#split-pattern-hierarchy

AudioFolder is a dataset builder to load an audio dataset with several thousand audio files. Additional information such as transcription is loaded by AudioFolder if its included in the metadata file

AudioFolder creates splits based on split pattern hierarchy 

```
# After structuring the data
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="/path/to/data")
```

**<u>Part 2: Push local dataset to Hub</u>**

https://huggingface.co/docs/datasets/upload_dataset

```
pip install huggingface_hub

huggingface-cli login

from datasets import load_dataset

dataset = load_dataset("stevhliu/demo")

dataset.push_to_hub("stevhliu/processed_demo")
```

<br/>
<br/>
<br/>

### <u>Approach 2</u>

https://huggingface.co/docs/datasets/audio_dataset#audiofolder

https://huggingface.co/docs/hub/datasets-adding

**<u>Part 1: Upload local dataset directory to Hub</u>**

**<u>Uploading Datasets in general</u>**

https://huggingface.co/docs/hub/datasets-adding

- Dataset repos are Git repos, so we can use Git to push data files to the Hub
- Starter: https://huggingface.co/docs/hub/repositories-getting-started
- Parquet is the recommended format due to its efficient compression etc.
    - For more general use cases involving analytics, data filtering or metadata parsing, Parquet is the recommended option for large scale image and audio datasets.
- For large scale image and audio datasets streaming, WebDataset should be preferred over raw image and audio files to avoid the overhead of accessing individual files
- Hugging Face Hub supports large scale datasets, usually uploaded in Parquet via push_to_hub() or WebDataset format

**<u>Creating audio datasets</u>**

- https://huggingface.co/docs/hub/datasets-audio
- https://huggingface.co/collections/datasets-examples/audio-dataset-66aca0b73e8f69e3d069e607

**<u>Uploading large folders</u>**

https://huggingface.co/docs/huggingface_hub/guides/upload#upload-a-folder-by-chunks

- Upload folder normally: ```upload_folder()```
    - Upload a local folder to an existing repo
    - Specify the path of the local folder to upload, where you want to upload the folder to in the repository, and the name of the repository you want to add the folder to. Depending on your repository type, you can optionally set the repository type as a dataset, model, or space

    ```
    from huggingface_hub import HfApi
    api = HfApi()

    api.upload_folder(
        folder_path="/path/to/local/space",
        repo_id="username/my-cool-space",
        repo_type="space",
    )
    ```

    - By default, the .gitignore file will be taken into account to know which files should be committed or not. By default we check if a .gitignore file is present in a commit, and if not, we check if it exists on the Hub. Please be aware that only a .gitignore file present at the root of the directory with be used. We do not check for .gitignore files in subdirectories.

    - Makes a single commit, fails explicitly when something wrong happens

- Upload a large folder: ```upload_large_folder()```
    - Resumable
        - Upload process is split into many small tasks
        - Each time a task is completed, result is cached locally in ```./cache/huggingface``` inside the folder you're trying to upload
    - Multi-threaded
    - Resilient to errors: High-level retry-mechanism
        - Downside: If transient errors happen, the process will continue and retry. If permanent errors happen (e.g. permission denied), it will retry indefinitely without solving the root cause.
    - Limitations
        - ...


    ```
    api.upload_large_folder(
        repo_id="HuggingFaceM4/Docmatix",
        repo_type="dataset",
        folder_path="/path/to/local/docmatix",
    )
    ```

- Recommendations
    - Start small

- Upload a folder by chunks: ```upload_folder()```
    - Upload a folder in serveral commits so we don't have to resume the process from the beginning: Pass ```multi_commits=True``` as a argument
    - Recommended to pass ```multi_commits_verbose=True```
    - Upload will resume from where it stopped
        - If the process is interrupted before completing, you can rerun your script to resume the upload. The created PR will be automatically detected and the upload will resume from where it stopped
    - ```multi_commits``` is still an experimental feature

**<u>Repo Limits and recommendations</u>**

https://huggingface.co/docs/hub/repositories-recommendations

- Repo size: Generally support repos up to 300GB
- Number of files: Keep total number of files under 100k
    - Large datasets can be exported as Parque files or in WebDataset format
    - Cannot exceed 10k files per folder. Solution is to create a repo structure that uses subdirectories 


**<u>Part 2: Load dataset from the hub using audiofolder</u>**

```
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="/path/to/data") # There's a streaming option: https://huggingface.co/docs/datasets/en/stream
```

### <u>Approach 3</u>

https://huggingface.co/docs/hub/repositories-getting-started

https://huggingface.co/docs/datasets/en/audio_dataset#loading-script ((Legacy) Loading script)

https://huggingface.co/docs/hub/datasets-audio

https://huggingface.co/collections/datasets-examples/audio-dataset-66aca0b73e8f69e3d069e607

https://medium.com/htx-dsai/finetuning-whisper-for-the-singaporean-home-team-context-a3ae1a6ae809

https://huggingface.co/docs/hub/datasets-webdataset

Custom loading script

Reasons
- For large scale image and audio datasets streaming, WebDataset should be preferred over raw image and audio files to avoid the overhead of accessing individual files. 
- Audio datasets are commonly stored in tar.gz archives which requires a particular approach to support streaming mode. 



<br/>
<br/>
<br/>

### Creating a dataset loading script for audio datasets

Audio datasets are commonly stored in tar.gz archives which requires a particular approach to support streaming mode

see ```new_dataset_script tutorial.py```

Step 1: Put the dataset into WebDataset format

vivos format:

```
- vivos.tar.gz
    - vivos.tar
        - train
            - genders.txt: Contains the gender type for each waves folder
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - VIVOSSPK01 -> Speaker ID
                    - VIVOSSPK01_R001.wav
                    - VIVOSSPK01_R002.wav
                    - VIVOSSPK01_R003.wav

            - test
    - prompts-train.txt.gz
        - prompts-train.txt: Contains transcriptions for all the .wav files
    - prompts-test.txt.gz
```

Usual size per archive is generally around 1GB?

```
- imda_nsc_p3.tar.gz
    - imda_nsc_p3.tar
        - train
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-1.tar
                    - 3000-1_1.wav
                    - 3000-1_2.wav
                    - 3000-1_3.wav
- prompts-train.txt.gz
    - prompts-train.txt: Contains transcriptions for all the train .wav files
```

try this first

```
- imda_nsc_p3.tar.gz
    - imda_nsc_p3.tar
        - train
            - prompts.txt: Contains transcriptions for all the .wav files
            - waves
                - 3000-1_1.wav
                - 3000-1_2.wav
                - 3000-1_3.wav
- prompts-train.txt.gz
    - prompts-train.txt: Contains transcriptions for all the train .wav files
```